**Replication of Aguiar & Gopinath (2007): Emerging Market Business Cycles: The cycle is the trend**

Empirical regularities + Matlab Codes

In [1]:
import pandas as pd #For data manipulation
import numpy as np #For matrices handling
import requests #For API interaction

#We will work with data from Mexico and Canada, 1980 to 2022, quarterly. Check for sources.
api_url = "http://stats.oecd.org/sdmx-json/data/QNA/MEX.GDP+B1_GE.CUR+VOBARSA.Q/all?startTime=1980-Q2&endTime=2011-Q4&dimensionAtObservation=allDimensions"
data = {"GDP":"","I":"","C":"","NX":"","I":"","X":""}
response = requests.get(api_url)
print(response.json())
#aux = pd.DataFrame.from_dict(response.json()[1:])
#aux.columns = response.json()[0] 


#Variables are: Output, Household Consumption (excl. Gov consumption), or private consumption
#Investment (as gross fixed capital accumulation), Net exports (X-I).

#Seasonal adjustment Using X12 ARIMA.

#Filterred of the series using HP 1600, (robust check with band pass freqs btw 6 and 32

#Moments of the RBC covered (using GMM, standard errores reported in parenthesis.)

#Exporting tables for authors contribution

#Figures for presentation.


{'header': {'id': '72832381-3c18-4955-a438-fe21d3b96157', 'test': False, 'prepared': '2023-02-20T00:31:05.367469Z', 'sender': {'id': 'OECD', 'name': 'Organisation for Economic Co-operation and Development'}, 'links': [{'href': 'https://stats.oecd.org:443/sdmx-json/data/QNA/MEX.GDP+B1_GE.CUR+VOBARSA.Q/all?startTime=1980-Q2&endTime=2011-Q4&dimensionAtObservation=allDimensions', 'rel': 'request'}]}, 'dataSets': [{'action': 'Information', 'observations': {'0:0:0:0:0': [8116813.7, 0, 0, 0, 0, 0], '0:0:0:0:1': [8292036.6, 0, 0, 0, 0, 0], '0:0:0:0:2': [8532162.9, 0, 0, 0, 0, 0], '0:0:0:0:3': [8722683.8, 0, 0, 0, 0, 0], '0:0:0:0:4': [8948701.0, 0, 0, 0, 0, 0], '0:0:0:0:5': [8995481.0, 0, 0, 0, 0, 0], '0:0:0:0:6': [9156642.0, 0, 0, 0, 0, 0], '0:0:0:0:7': [8980213.0, 0, 0, 0, 0, 0], '0:0:0:0:8': [8993897.2, 0, 0, 0, 0, 0], '0:0:0:0:9': [8929212.7, 0, 0, 0, 0, 0], '0:0:0:0:10': [8733285.4, 0, 0, 0, 0, 0], '0:0:0:0:11': [8668713.5, 0, 0, 0, 0, 0], '0:0:0:0:12': [8494410.4, 0, 0, 0, 0, 0], '0:0:0:0